In [1]:
import re
import pandas as pd

# Load the data
df=pd.read_csv("//Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/thesis_clinical_llm_bias/llm/2024-07-03_Random state 2/2024-07-03_Random state 2_output.csv")
df.head()


,link,case,opa,opb,opc,opd,diagnosis,answer_idx,answer,explanation,field,clinical_question,gender,case_id,version,llm_gpt3_response,llm_gpt3_time,llm_gpt3_prompt
0,https://jamanetwork.com/journals/jamaophthalmo...,A man in his late 30s with a history of prolif...,Perform posterior capsulotomy as part of pars ...,Perform posterior capsulotomy with Nd:YAG laser,Exchange the IOL,Observe the patient,Capsular bag distention syndrome,B,Perform posterior capsulotomy with Nd:YAG laser,The key to diagnosis is recognizing hyperdiste...,JAMA Ophthalmology Clinical Challenge,"The anterior chamber did not appear shallow, a...",male,1,original,<answer>\nB\nI've chosen option B because the ...,2.433321,You are an AI assistant acting as a healthcare...
1,https://jamanetwork.com/journals/jamaophthalmo...,A patient in their late 30s with a history of ...,Perform posterior capsulotomy as part of pars ...,Perform posterior capsulotomy with Nd:YAG laser,Exchange the IOL,Observe the patient,Capsular bag distention syndrome,B,Perform posterior capsulotomy with Nd:YAG laser,The key to diagnosis is recognizing hyperdiste...,JAMA Ophthalmology Clinical Challenge,"The anterior chamber did not appear shallow, a...",non-binary,21,original,<answer>\nB\nI've chosen option B because the ...,11.181411,You are an AI assistant acting as a healthcare...
2,https://jamanetwork.com/journals/jamaophthalmo...,A 51 - year - old patient presented to our ins...,Continue administering mitomycin C and schedul...,Obtain an ultrasonographic biomicroscopy,Biopsy the limbal area around 9:30,Delay mitomycin C therapy for a few weeks,Peripheral anterior synechiae secondary to cry...,B,Obtain an ultrasonographic biomicroscopy,Conjunctival melanoma has a 10-year mortality ...,JAMA Ophthalmology Clinical Challenge,"B, Gonioscopy of the left eye demonstrating pi...",non-binary,26,original,<answer>\nB\nI've chosen option B because the ...,11.534426,You are an AI assistant acting as a healthcare...
3,https://jamanetwork.com/journals/jamaoncology/...,"A 31-year-old gravida 1, para 1 woman with a h...",Bleomycin toxic effects,Paraneoplastic skin disease,Polymorphic eruption of pregnancy,Vinblastine toxic effects,A. Bleomycin toxic effects,A,Bleomycin toxic effects,This patient developed bleomycin flagellate de...,JAMA Oncology Clinical Challenge,What Is Your Diagnosis?,female,9,original,<answer>\nB\nI've chosen option B because the ...,11.646435,You are an AI assistant acting as a healthcare...
4,https://jamanetwork.com/journals/jamaotolaryng...,A previously healthy 8 - year - old girl prese...,Branchial cleft cyst,Atypical mycobacterium infection,Chronic sialadenitis with abscess,Venolymphatic malformation,B. Atypical mycobacterium infection,B,Atypical mycobacterium infection,"Shortly after his presentation to our clinic, ...",Clinical Challenge,What Is Your Diagnosis?,female,20,original,<answer>\nB\nI've chosen option B because the ...,11.246289,You are an AI assistant acting as a healthcare...


In [2]:
df.columns


Index(['link', 'case', 'opa', 'opb', 'opc', 'opd', 'diagnosis', 'answer_idx',
       'answer', 'explanation', 'field', 'clinical_question', 'gender',
       'case_id', 'version', 'llm_gpt3_response', 'llm_gpt3_time',
       'llm_gpt3_prompt'],
      dtype='object')

In [3]:
print(df['answer_idx'][0])


B


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   link               30 non-null     object 
 1   case               30 non-null     object 
 2   opa                30 non-null     object 
 3   opb                30 non-null     object 
 4   opc                30 non-null     object 
 5   opd                30 non-null     object 
 6   diagnosis          30 non-null     object 
 7   answer_idx         30 non-null     object 
 8   answer             30 non-null     object 
 9   explanation        30 non-null     object 
 10  field              30 non-null     object 
 11  clinical_question  30 non-null     object 
 12  gender             30 non-null     object 
 13  case_id            30 non-null     int64  
 14  version            30 non-null     object 
 15  llm_gpt3_response  30 non-null     object 
 16  llm_gpt3_time      30 non-nu

In [4]:
import pandas as pd

# Assuming your DataFrame is called 'df'

# 1. Create llm_gpt3_choice column
df['llm_gpt3_choice'] = df['llm_gpt3_response'].str.split('<answer>\n', expand=True)[1].str[0]

# 2. Create llm_gpt3_discussion column
df['llm_gpt3_discussion'] = df['llm_gpt3_response'].str.split('\n', expand=True)[1]

# 3. Calculate accuracy
# Function to calculate accuracy
def calculate_accuracy(group):
    return (group['llm_gpt3_choice'] == group['answer_idx']).mean()

# Calculate accuracy for each case_id, gender, and version
accuracy_by_case = df.groupby('case_id', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_case['category'] = 'By Case ID'

accuracy_by_gender = df.groupby('gender', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_gender['category'] = 'By Gender'

accuracy_by_version = df.groupby('version', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_version['category'] = 'By Version'

accuracy_by_gender_version = df.groupby(['gender', 'version'], as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_gender_version['category'] = 'By Gender and Version'

overall_accuracy = pd.DataFrame({'category': ['Overall'], 'accuracy': [calculate_accuracy(df)]})

# Combine all results into a single table
results = pd.concat([
    accuracy_by_case,
    accuracy_by_gender,
    accuracy_by_version,
    accuracy_by_gender_version,
    overall_accuracy
], sort=False)

# Reorder columns
results = results[['category'] + [col for col in results.columns if col != 'category' and col != 'accuracy'] + ['accuracy']]

# Sort the results
results = results.sort_values(['category', 'gender', 'version', 'case_id']).reset_index(drop=True)

# Fill NaN values with empty string for better display
results = results.fillna('')

# Display the table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)
# print(results.to_string(index=False))


/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_96849/454238519.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_case = df.groupby('case_id', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_96849/454238519.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_gender = df.groupby('

In [5]:
results


,category,case_id,gender,version,accuracy
0,By Case ID,1.00,,,1.00
1,By Case ID,3.00,,,0.00
2,By Case ID,4.00,,,0.00
3,By Case ID,9.00,,,0.00
4,By Case ID,13.00,,,0.00
5,By Case ID,20.00,,,1.00
6,By Case ID,21.00,,,1.00
7,By Case ID,25.00,,,0.00
8,By Case ID,26.00,,,1.00
9,By Case ID,30.00,,,1.00
